In [1]:
###### Importo librerias y leo el archivo a trabajar  ######
from fastapi import FastAPI
import pandas as pd
import numpy as np
import locale
import ast
df = pd.read_csv("DataSets/movies_dataset.csv",low_memory=False)

###### Creo dos funciones que me permiten desempaquetar los datos de tipo diccionarios y listas #######
def desempaquetarlista(obj): 
    if isinstance(obj, str) and '{' in obj:
        L = []
        for i in ast.literal_eval(obj):
            L.append(i['name'])
        return L

def desempaquetardicc(obj): 
    if isinstance(obj, str) and '{' in obj:
        dic = ast.literal_eval(obj)
        return dic['name']
    
######  Aplico las funciones antes mencionadas a cada columna y sobreescribo el dataframe #######
df["genres"] = df["genres"].apply(desempaquetarlista)
df["belongs_to_collection"] = df["belongs_to_collection"].apply(desempaquetardicc)
df["production_companies"]  = df["production_companies"].apply(desempaquetarlista)
df["production_countries"]  = df["production_countries"].apply(desempaquetarlista)
df["spoken_languages"] = df["spoken_languages"].apply(desempaquetarlista)

###### Convierto el campo release_date en datetime y le doy formato luego creo una mascara para eliminar los nulos  ######
df ['release_date'] = pd.to_datetime(df["release_date"], format='%Y-%m-%d', errors = 'coerce')
mascara = df[df["release_date"].isnull()]
df = df.drop(labels= mascara.index, axis=0)

###### Convierto los campos revenue y budget en numericos y genero la nueva columna return ######
df["revenue"] = pd.to_numeric(df["revenue"], errors="coerce")
df["budget"] = pd.to_numeric(df["budget"], errors="coerce")
df["return"] = df["revenue"] / df["budget"]


###### Reemplazo los valores +infinito -infinito por nulos luego a los nulos los convierto en 0 y finalmente redondeo los numeros #######
df["return"] = df["return"].replace([np.inf, -np.inf], np.nan)
df["return"] = df["return"].fillna(0)
df["return"] = df["return"].round(0)

###### Elimino las columnas que me marca el proyecto ######
df.drop("video",axis="columns",inplace=True)
df.drop("imdb_id",axis="columns",inplace=True)
df.drop("adult",axis="columns",inplace=True)
df.drop("original_title",axis="columns",inplace=True)
df.drop("vote_count",axis="columns",inplace=True)
df.drop("poster_path",axis="columns",inplace=True)
df.drop("homepage",axis="columns",inplace=True)

###### Genero dos columnas nuevas con el mes y el dia en formato texto en español ######
df["month_name"] = df["release_date"].dt.month_name(locale.setlocale(locale.LC_TIME, 'es_ES'))
df["day_name"] = df["release_date"].dt.day_name(locale.setlocale(locale.LC_TIME, 'es_ES'))

###### Finalmente terminado las modificaciones y el ETL generando un nuevo csv limpio para el consumo de la api ######
df.to_csv("DataSets/Peliculas.csv",index=False)
